In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# =========================
# GOOGLE DRIVE PATHS
# =========================
BASE_PATH = "/content/drive/MyDrive/Chatbot"
IMG_DIR = BASE_PATH + "/DeepWeeds/images/"
LABEL_DIR = BASE_PATH + "/DeepWeeds/labels/"
MODEL_SAVE_DIR = BASE_PATH + "/"

BATCH_SIZE = 32
IMG_SIZE = (224, 224)
INPUT_SHAPE = (224, 224, 3)
CLASSES = [0,1,2,3,4,5,6,7,8]
CLASS_STR = [str(i) for i in CLASSES]

# =========================
# LOAD FOLD 0
# =========================
train_df = pd.read_csv(LABEL_DIR + "train_subset0.csv")
val_df   = pd.read_csv(LABEL_DIR + "val_subset0.csv")

# Convert labels to string
train_df["Label"] = train_df["Label"].astype(str)
val_df["Label"] = val_df["Label"].astype(str)

# =========================
# DATA GENERATORS
# =========================
train_gen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    train_df,
    IMG_DIR,
    x_col="Filename",
    y_col="Label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    classes=CLASS_STR
)

val_gen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    val_df,
    IMG_DIR,
    x_col="Filename",
    y_col="Label",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    classes=CLASS_STR
)

# =========================
# BUILD RESNET50
# =========================
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=INPUT_SHAPE
)

# Freeze base model
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(len(CLASSES), activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# =========================
# SAVE MODEL EVERY EPOCH
# =========================
checkpoint = ModelCheckpoint(
    filepath=MODEL_SAVE_DIR + "resnet_epoch_{epoch:02d}.h5",
    save_freq="epoch",
    save_best_only=False
)

# =========================
# TRAIN
# =========================
model.fit(
    train_gen,
    epochs=50,
    validation_data=val_gen,
    callbacks=[checkpoint]
)

# Save final model
model.save(MODEL_SAVE_DIR + "resnet_final.h5")


Found 9085 validated image filenames belonging to 9 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1416 invalid image filename(s) in x_col="Filename". These filename(s) will be ignored.
  warnings.warn(


Found 3087 validated image filenames belonging to 9 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 414 invalid image filename(s) in x_col="Filename". These filename(s) will be ignored.
  warnings.warn(


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


...

284/284 ━━━━━━━━━━━━━━━━━━━━ 53s 188ms/step - accuracy: 0.4611 - loss: 1.5720 - val_accuracy: 0.4645 - val_loss: 1.5693
Epoch 37/50
284/284 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.4580 - loss: 1.5724

284/284 ━━━━━━━━━━━━━━━━━━━━ 53s 186ms/step - accuracy: 0.4580 - loss: 1.5724 - val_accuracy: 0.4603 - val_loss: 1.5682
Epoch 38/50
284/284 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.4428 - loss: 1.5887

284/284 ━━━━━━━━━━━━━━━━━━━━ 52s 183ms/step - accuracy: 0.4428 - loss: 1.5886 - val_accuracy: 0.4661 - val_loss: 1.5667
Epoch 39/50
284/284 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.4575 - loss: 1.5778

284/284 ━━━━━━━━━━━━━━━━━━━━ 53s 185ms/step - accuracy: 0.4575 - loss: 1.5778 - val_accuracy: 0.4645 - val_loss: 1.5639
Epoch 40/50
284/284 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.4503 - loss: 1.5800

KeyboardInterrupt: 

In [ ]:
print(train_gen.class_indices)
